In [1]:
from os import listdir
from os.path import isfile, join
from PIL import Image, ImageDraw, ImageFont
import cv2
import numpy as np

In [2]:
def read_resized(path, final_height, final_width):
    img_read = Image.open(path)  #leer imagen
    img_resized = img_read.resize((final_height,final_width))
    return img_resized

def create_file_path(path, type):
    #type = real or generated
    path_frame_original = [arch for arch in listdir(path) if isfile(join(path, arch))] 
    if(type == 'real'):
        path_ok = [str(x) + '.jpg' for x in range(len(path_frame_original))]
    if(type == 'generated'):
        path_ok = [str(x) + '.png' for x in range(len(path_frame_original))]
    return path_ok

def escala_de_grises(im):
    '''
    get from: https://wizardprogrammer.blogspot.com/2019/03/convertir-imagen-blanco-y-negro-python.html
    '''
    #dimension imagen
    x = im.size[0]
    y = im.size[1]
    #Creamos una nueva imagen con las dimensiones de la imagen anterior
    im2 = Image.new('RGB', (x, y))
    i = 0
    while i < x:
        j = 0
        while j < y:
            #Obtenemos el valor RGB de cada pixel
            r, g, b = im.getpixel((i,j))
            #Obtenemos su equivalente en la escala de gris
            p = (r * 0.3 + g * 0.59 + b * 0.11)
            #Ese valor lo convertimos a entero
            gris = int(p)
            pixel = tuple([gris, gris, gris])
            #En la nueva imagen en la posición i, j agregamos el nuevo color 
            im2.putpixel((i,j), pixel)
            j += 1
        i += 1
    return im2

def add_blank_space_fixed(img_B_W ,random_blank, h_blank = 50 , w_blank = 50 , h_original =256, w_original=256, pos_x_blank = 0, pos_y_blank = 0):  
    '''
    Agregar espacio en blanco a la imagen input blanco y negro.
    img_B_W: imagen a extraer espacio en blanco
    random_blank: True o False, si se quiere un recorte random o no
    h_blank: tamaño horizonal del recorte
    w_blank: tamaño vertical del recorte
    h_original: tamaño de la imagen
    w_original: tamaño de la imagen
    pos_x_blank: opcional si se elegio un recorte no aleatorio, dar coordenada x al espacio en blanco
    pos_y_blank: opcional si se elegio un recorte no aleatorio, dar coordenada x al espacio en blanco
    ''' 
    img_white = Image.new('RGB', (h_blank, w_blank), "white")   
    if random_blank == True:  #recorte en blanco random
        pos_h_random = np.random.randint(h_original - h_blank)  
        pos_w_random = np.random.randint(w_original - w_blank)    
    if random_blank == False:  #coordenadas del recorte
        pos_h_random = pos_x_blank
        pos_w_random = pos_y_blank
    img_B_W.paste(img_white,(pos_h_random, pos_w_random))
    return img_B_W

def set_title(img, text, color):
    draw = ImageDraw.Draw(img)  #abrir dibujo
    font = ImageFont.truetype("arial.ttf", 20)
    draw.text((10, 10), text, font=font, fill=color)
    return img

def create_video(path_folder_real, path_folder_generated, real, generated, input_height, input_widht, pathOut, fps, blank, random_blank, blank_x, blank_y, blank_height, blank_widht):
    frame_array = []   #array con las matrices numpy de cada frame
    for i in range(len(real)):   #puede ser real o generated, misma cantidad de imagenes
        #procesar imagenes
        real_image = read_resized(path_folder_real + real[i], input_height, input_widht)
        generated_image = read_resized(path_folder_generated + generated[i], input_height, input_widht)
        input_bw = escala_de_grises(real_image)
        if blank == True:
            input_bw = add_blank_space_fixed(input_bw, random_blank, blank_height, blank_widht, input_height, input_widht, blank_x, blank_y)
        
        #titulos
        set_title(input_bw, 'input', 'white')
        set_title(real_image, 'real', 'white')
        set_title(generated_image, 'generated', 'white')                
        
        #unir imagenes
        list_images = [input_bw, real_image, generated_image]
        final_img = Image.new('RGB', (input_widht * 3, input_height))     
        x_offset = 0
        for im in list_images:
            final_img.paste(im, (x_offset,0))
            x_offset += im.size[0]
        #tranformar la imagen pillow a cv2   
        final_img = cv2.cvtColor(np.array(final_img), cv2.COLOR_BGR2RGB) 
        
        #unir a un array para crear video con cv2
        frame_array.append(final_img)
    
    #crear video
    height_f, width_f, _ = final_img.shape
    size = (width_f,height_f)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size) #create a file video 
    for i in range(len(frame_array)):
        out.write(frame_array[i])            
    out.release()

In [3]:
print('Crear video a través de los frames originales sin procesar y de los frames generador por la red pix2pix')

Crear video a través de los frames originales sin procesar y de los frames generador por la red pix2pix


In [4]:
##PARAMETROS
#path de los frames originales sin procesar
path_real_folder = 'E:/Desktop/MLBI - 2020/Proyecto GANs/Data animals/Frames_original/dog select.mp4/'  

#path de los frames generados por pix2pix
path_generated_folder = 'E:/Desktop/MLBI - 2020/Proyecto GANs/Data videos-to-pix2pix-procesed/dog select.mp4/'

#definir Nombre de archivo donde se va a guardar el video tras unir los frames
path_folder_video = 'E:/Desktop/MLBI - 2020/Proyecto GANs/Data videos-released/'
name_video_file = 'dog select.mp4'
path_video = path_folder_video + name_video_file

#Parametros
height = 256  
width = 256   

In [5]:
real_file = create_file_path(path_real_folder, type = 'real')  #lista con los valores reales

generated_file = create_file_path(path_generated_folder, type = 'generated')

create_video(   path_folder_real = path_real_folder,
                path_folder_generated = path_generated_folder,
                real = real_file,
                generated = generated_file,
                input_height = height,
                input_widht = width,
                pathOut = path_video,
                fps = 15,
                blank = True,   #agregar un cuadro blanco al input
                random_blank = False,   #cuadro blanco agregado random?
                blank_x = 100,  #posicion x del cuadro blanco
                blank_y = 100,
                blank_height = 77,  #ancho del cuadro banco
                blank_widht = 77
                )